# Sesión 7.1 BERT/RoBERTa y contextual word embeddings

En esta sesión se muestra un ejemplo de funcionamiento de los word embeddings contextuales de los Transformers como BERT o RoBERTa.

En esta sesión:
1.  Instalaremos las librerías de transformers y datasets.
2.  Descargaremos un modelo BERT y un modelo RoBERTa preeentrenados.
3.  Veremos el funcionamiento del tokenizador WordPiece usado por BERT y del tokenizer BPE usado por RoBERTa.
4.  Con el modelo BERT: (1) veremos la representación (embedding) de una misma palabra ('banco') en diferentes contextos (textos) y (2) calcularemos la similitud de estas representaciones.
5.  Haremos lo mismo usando el modelo RoBERTa.



In [1]:
# Install libraries
!pip3 install -U transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 

## Apartado 1.1 Descargamos el modelo de BERT preentrenado

Descargamos un modelo de BERT preentrenado como BETO o mBERT.

Cada word embedding de BERT está representado por vectores de 768 características (dimensiones) para los modelos *base* y 1024 en los modelos *large*.

In [2]:
import transformers

# Modelo de BETO
path_beto_model = 'dccuchile/bert-base-spanish-wwm-uncased'
# Modelo de multilingual BERT
path_bert_multiligual_model = 'bert-base-multilingual-cased'
# Modelo de distilbert en español
path_bert_sp_model = 'CenIA/distillbert-base-spanish-uncased'
# Modelo de RoBERTa de MarIA
path_roberta_model = 'PlanTL-GOB-ES/roberta-base-bne'

# Obtenemos el tokenizer y el modelo de BERT
bert_sp_tokenizer = transformers.AutoTokenizer.from_pretrained (path_bert_sp_model, use_fast=True)
bert_sp_model = transformers.AutoModel.from_pretrained (path_bert_sp_model, output_hidden_states=True)

# Obtenemos el tokenizer y el modelo de RoBERTa
roberta_tokenizer = transformers.AutoTokenizer.from_pretrained (path_roberta_model, use_fast=True)
roberta_model = transformers.AutoModel.from_pretrained (path_roberta_model, output_hidden_states=True)

tokenizer = bert_sp_tokenizer
model = bert_sp_model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Apartado 1.2 WordPiece tokenizer de BERT y BPE tokenizer con RoBERTa

BERT usa un WordPieceTokenizer que es un tokenizador subpalabra. Durante la tokenización, se segmenta cada palabra según las subpalabras más largas presentes en el vocabulario (estrategia greedy, derecha a izquierda o izquierda a derecha). Las subpalabras que no comienzan una palabra llevan un prefijo especial, comúnmente ##.

Por ejemplo, Palabra: unbelievable ---> Tokens: ["un", "##believ", "##able"]

De esta forma tiene en cuenta las inflexiones de las palabras como sufijos y conjugaciones verbales, manteniendo un vocabulario de tamaño razonable.

BERT utiliza los siguientes tokens especiales:
* [UNK] – Unknown token. Representa a tokens que no están en el vocabulario y que por tanto no pueden convertirse a ningún id.
* [SEP] – Separator token. Usado para separar secuencias de texto para tareas de clasificación o de pregunta-respuesta.
* [PAD] – Padding. Este token se utiliza de relleno en textos de distinto tamaño hasta completar el tamaño deseado (es mucho más eficiente procesar textos cuando todos tienen el mismo tamañao).
* [CLS] – Classifier token. Se usa para la clasificación del fragmento de texto. Es el primer token del texto, que marca su inicio.
* [MASK] – Masking. Se utiliza para enmascarar la palabra que se quiere predecir cuando se entrena un modelo enmascarando palabras.

Los modelos RoBERTa utilizan el tokenizer BPE (Byte-Pair Encoding). En la implementación de BPE usada en RoBERTa se añade un espacio en blanco explícito al principio de las palabras (p.e: 'Ġlenguaje'), de forma que el espacio se vuelve parte de la codificación y no hace falta un paso específico para determinar el comienzo de las palabras.

Los tokens especiales de BPE en RoBERTa son los siguientes:
* \<s> Para indicar el inicio de la frase
* \</s> Para indicar el fin de la frase

In [3]:
texto = "Estudiaré la asignatura de procesamiento del lenguaje natural escrito y terminaré las prácticas en casa."

print("texto crudo ---> " + texto)
print("-------------"*15)
print("Tokenizacion con WordPiece-BERT")

tokenizer = bert_sp_tokenizer
tokens = tokenizer.tokenize(texto, add_special_tokens=True)
print(f"Tokens bert ---> {tokens}")
print(f"Ids bert    ---> {tokenizer.convert_tokens_to_ids(tokens)}")
print(f"texto       ---> {tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens))}")
print("-------------"*15)

print("Tokenizacion con BPE-RoBERTa")
tokenizer = roberta_tokenizer
tokens = tokenizer.tokenize(texto, add_special_tokens=True)
print(f"Tokens RoBERta ---> {tokens}")
print(f"Ids RobERTa    ---> {tokenizer.convert_tokens_to_ids(tokens)}")
print(f"texto          ---> {tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens))}")
print("-------------"*15)


texto crudo ---> Estudiaré la asignatura de procesamiento del lenguaje natural escrito y terminaré las prácticas en casa.
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Tokenizacion con WordPiece-BERT
Tokens bert ---> ['[CLS]', 'estudiar', '##é', 'la', 'asigna', '##tura', 'de', 'procesamiento', 'del', 'lenguaje', 'natural', 'escrito', 'y', 'terminar', '##é', 'las', 'prácticas', 'en', 'casa', '.', '[SEP]']
Ids bert    ---> [4, 6661, 30982, 1032, 12373, 1571, 1009, 14169, 1081, 8023, 2901, 4749, 1040, 5000, 30982, 1085, 4636, 1035, 1635, 1008, 5]
texto       ---> [CLS] estudiaré la asignatura de procesamiento del lenguaje natural escrito y terminaré las prácticas en casa. [SEP]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Apartado 1.3 Word embeddings contextuales de BERT

BERT tiene distintas capas ocultas dependiendo del modelo *base* o *large* preentrenado. En los modelos *base* el número de capas (encoders) es 12 y en los modelos *large* 24, que junto con la capa de entrada forman 13 y 25 capas respectivamente.

La capa de entrada se situa justo antes del primer encoder, es decir justo antes de la primera capa de atención y convierte la secuencia de tokens en una matriz de embeddings.

Texto --> Tokenización --> [Capa de Entrada] --> [Bloques Transformers (12 ó 24] --> Salidas.

Para cada token de nuestra frase tendremos entonces 13 capas de vectores de 768 características.

**Los embeddings de cada token son distintos y dependen del contexto donde aparezcan en el texto**.

En el código siguiente se muestran ejemplos en los que se obtienen los distintos vectores de la misma palabra "banco" según su contexto en diferentes oraciones y se comparan (similitud) entre sí.

Para el cálculo del word embedding de la palabra utilizamos las 4 últimas capas y sumamos sus valores. Para ello definiremos dos funciones, una para el cálculo del embedding y otra

OBSERVACIÓN: Existen otras estrategias para el cálculo del embedding de salida, que se utilizan en función de la tarea a realizar:

*   Clasificación:  [CLS] embedding
*   Similitud de frases: Mean Pooling o capa intermedia
*   Indexación/Búsqueda: Media o media de capas finales.
*   Traducción/Generación: token embeddings individuales.
*   Embeddings genéricos: Media de las 4 últimas capas (Sentence-BERT style)

In [4]:
# Funciones para obtener la media de las 4 últimas capas correspondientes a un
# token determinado de una frase.

# Conjuntamente:
# get_word_vector, tokeniza una sentencia (sent) de acuerdo con un tokenizador
# (tokenizer) e identifica los tokens correpondientes a una palabra dada de
# la sentencia (identificada por idx); después llama a get_hidden_states para
# obtener una codificación de dicha palabra en la sentencia.

import numpy as np
import torch
from scipy import spatial
from transformers import AutoTokenizer, AutoModel

def get_hidden_states(encoded, token_ids_word, model, layers=None):

    """Introduce una sentencia en el modelo y a la salida apila y suma los
       tokens de las cuatro últimas capas correspondientes a la palabra de
       interés, identificados por sus ids, y devuelve su media.

       @encoded: input sentence, once tokenized for being input for the model.
       @tokens_ids_word: list of sub-tokens indices corresponding to the word
                         we want to get the hidden states for.
       @model: petrained model
       @layers: layers to get. Default: last four.
    """

    with torch.no_grad():
        output = model(**encoded, output_hidden_states=True)

    # Get all hidden states
    states = output.hidden_states

    # Use specified layers or default to the last four
    if layers is None:
        layers = [-1]  # Default: last four layers
    states = [states[i] for i in layers]

    # Stack and sum the selected layers
    output = torch.stack(states).sum(0).squeeze()

    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]

    # Return average over sub-tokens
    return word_tokens_output.mean(dim=0)


def get_word_vector(sent, idx, tokenizer, model, layers=None):

    """Get a word vector by first tokenizing the input sentence, getting all token idxs
       that make up the word of interest, and then `get_hidden_states`."""

    # Tokenize the input sentence
    encoded = tokenizer(sent, return_tensors="pt", add_special_tokens=False)

    # Get all token indices that belong to the word of interest
    word_ids = encoded.word_ids()
    token_ids_word = np.where(np.array(word_ids) == idx)[0]

    if len(token_ids_word) == 0:
        raise ValueError(f"No tokens found for word at index {idx}.")

    return get_hidden_states(encoded, token_ids_word, model, layers=layers)

### Usamos la función get_word_vector para obtener el embedding contextual BERT de 'banco' en un texto y lo mostramos.

Más adelante usaremos esta función sobre diferentes textos para mostrar la similitud de los vectores obtenidos mediante la misma.

In [5]:
textos = ["He pedido un préstamo al banco y me ha dado un interés del 5%.",
          "Vamos a sentarnos en ese banco del parque.",
          "Estoy cansado y quiero acostarme en ese banco.",
          "¿Sabes donde está el banco que tengo que sacar dinero?"]

token = 'banco'

tokenizer = bert_sp_tokenizer
model = bert_sp_model

idx=tokenizer.tokenize(textos[0]).index(token)
word_embedding = get_word_vector(textos[0], idx, tokenizer, model)
print('El vector del token ',token,' es: ', word_embedding)
print('La dimensión del vector es', word_embedding.shape)


El vector del token  banco  es:  tensor([-6.3169e-03,  1.8703e-01,  3.6935e-01,  3.0131e-01,  9.8954e-01,
         1.0901e-01, -6.5430e-02,  2.5144e-01, -4.3761e-01,  5.9788e-02,
        -5.5280e-01, -5.5382e-01,  1.0355e-01,  1.1549e-02, -2.3408e-02,
         5.7747e-01, -6.1917e-01, -3.8683e-01, -9.1462e-03,  5.8836e-01,
         3.0504e-01, -8.1601e-01, -7.6535e-02,  5.4490e-01, -7.6231e-02,
        -1.2834e-01,  1.3908e-01,  2.1302e-01, -4.5684e-01, -1.6028e-01,
         1.0431e-01, -5.8983e-03, -7.1799e-01,  4.5879e-01, -4.7348e-02,
         4.3973e-02,  1.2724e-01, -3.9996e-01,  1.2190e+00,  6.4978e-01,
         1.4601e-01, -1.8018e-01,  4.0012e-01,  1.5588e-01,  4.2114e-01,
        -1.8565e-01, -2.4705e-01, -4.0832e-02,  5.1615e-01, -1.4808e-01,
         7.1355e-03,  1.1991e-02, -4.9200e-02,  7.5935e-01,  3.4633e-01,
        -8.7018e-01, -2.3004e-02, -3.6076e-01,  2.5623e-01,  3.0274e-01,
        -9.0575e-01,  3.3557e-02, -2.9256e-01,  1.9933e-01, -9.1272e-01,
         1.0300e-0

### Calculamos similitudes de 'banco' en diferentes *textos*

In [6]:
# Primero, encapsulamos el código a usar en una función general que dados
# un conjunto de textos y una palabra nos permita mostrar las similitudes
# de la palabra en dichos textos.

def show_similarities(textos, token, tokenizer, model):
  # Calculamos los word embeddings de word para los diferentes textos.
  word_embeddings = []
  for i in range(len(textos)):
    idx=tokenizer.tokenize(textos[i]).index(token)
    word_embeddings.append(get_word_vector(textos[i], idx, tokenizer, model))
    print(f"textos[{i}] ---> {textos[i]}")

  print()

  # Calculamos las similitudes entre los tokens de word en los distintos textos.
  for i in range(len(textos)):
    for j in range(i+1,len(textos)):
      similarity = 1 - spatial.distance.cosine (word_embeddings[i],
                                                word_embeddings[j])
      print(f"similitud de {token} en {i} y {j} ---> {1 - spatial.distance.cosine(word_embeddings[i], word_embeddings[j])}")


In [7]:
# calculamos similitudes de 'banco' en los diferentes textos utilizando
# el tokenizador y el modelo de BERT
tokenizer = bert_sp_tokenizer
model = bert_sp_model
token = 'banco'
show_similarities(textos, token, tokenizer, model)

textos[0] ---> He pedido un préstamo al banco y me ha dado un interés del 5%.
textos[1] ---> Vamos a sentarnos en ese banco del parque.
textos[2] ---> Estoy cansado y quiero acostarme en ese banco.
textos[3] ---> ¿Sabes donde está el banco que tengo que sacar dinero?

similitud de banco en 0 y 1 ---> 0.6427252888679504
similitud de banco en 0 y 2 ---> 0.7095755934715271
similitud de banco en 0 y 3 ---> 0.7737430930137634
similitud de banco en 1 y 2 ---> 0.8837725520133972
similitud de banco en 1 y 3 ---> 0.7034050226211548
similitud de banco en 2 y 3 ---> 0.7617104053497314


## Apartado 1.4 Word embeddings contextuales de RoBERTA

Igual que antes, pero usando el modelo RoBERTa con su tokenizador correspondiente.


In [8]:
tokenizer = roberta_tokenizer
model = roberta_model
token = 'Ġbanco'

show_similarities(textos, token, tokenizer, model)

textos[0] ---> He pedido un préstamo al banco y me ha dado un interés del 5%.
textos[1] ---> Vamos a sentarnos en ese banco del parque.
textos[2] ---> Estoy cansado y quiero acostarme en ese banco.
textos[3] ---> ¿Sabes donde está el banco que tengo que sacar dinero?

similitud de Ġbanco en 0 y 1 ---> 0.7344953417778015
similitud de Ġbanco en 0 y 2 ---> 0.8155832886695862
similitud de Ġbanco en 0 y 3 ---> 0.899459719657898
similitud de Ġbanco en 1 y 2 ---> 0.7873343229293823
similitud de Ġbanco en 1 y 3 ---> 0.7709649205207825
similitud de Ġbanco en 2 y 3 ---> 0.8234519362449646


Como se puede observar, se obtiene una representación distinta para cada palabra según el contexto. Los embeddings contextuales codifican la palabra en función de su significado en un contexto, puesto que este contexto cambia tambie´n cambia el embedding.
Obsérvese que si el contexto no es lo suficientemente grande el embedding puede no representar fielmente el significado de la palabra.